**Objective**: Convert natural language queries into SQL using an appropriate pre-trained model.

**Suggested Models**:
tscholak/optimum-**nl2sql**

In [1]:
import sqlite3
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model
tokenizer = AutoTokenizer.from_pretrained("tscholak/cxmefzzi")
model = AutoModelForSeq2SeqLM.from_pretrained("tscholak/cxmefzzi")

# convert NL to SQL
def nl_to_sql(nl_query, db_id, schema):
    input_text = f"{nl_query} | {db_id} | {schema}"
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=128)
    sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql

# Clean model output
def clean_predicted_sql(predicted_sql):
    if "|" in predicted_sql:
        parts = predicted_sql.split("|")
        if len(parts) > 1:
            return parts[1].strip()
    return predicted_sql.strip()


def create_sample_db():
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE customers (id INTEGER, name TEXT, city TEXT, age INTEGER);")
    sample_data = [
        (1, 'Aditya', 'New York', 25),
        (2, 'Aman', 'japan', 35),
        (3, 'Ayush', 'New York', 40),
        (4, 'Pari', 'Mumbai', 30),
        (5, 'Kadu', 'Mumbai', 25),
    ]
    cursor.executemany("INSERT INTO customers VALUES (?, ?, ?, ?);", sample_data)
    conn.commit()
    return conn


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/tscholak/cxmefzzi/104b1aadc4dc960a05d79ac7d9826e3407ad46705daa27927d2ff43d762b13ed?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1747827388&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzgyNzM4OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby90c2Nob2xhay9jeG1lZnp6aS8xMDRiMWFhZGM0ZGM5NjBhMDVkNzlhYzdkOTgyNmUzNDA3YWQ0NjcwNWRhYTI3OTI3ZDJmZjQzZDc2MmIxM2VkP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=U72RWz9wZM0QPvn3-dcG-u4ZEdSJwv-PdL9IzqU1v-PQ-B6m6yoxTDbk8lcgCyBqxMv0rA4YEKXR-t6ypCE459Cb9pCxW3j0G3LdJwdwa4tE7vuRGNIZ9AcUYci62PfEJJRO0lx7PY%7Ex1KFCtipt3q%7EobBdTJaa2-jRFxUpmsu-WkVNxgAYCJnpePt%7Eg%7Em2LCzvdQo-chFMX9wFgyq0AS62wOsbWnmknE1oqSfWwhNP8x%7EAzWoaOG1qABiBS1KyRTLBMHhGQqLwlZw2XFi7Q6-0c%7ETWGxDscwwxIWSfcDUVFlzpSyqKmgqR-3gXTXV8a%7EejzhrbkG2QgUH5vdtGfRw__&Key-Pair-Id=K3RPWS32NSSJCE: HTTPSConnectionPool(h

model.safetensors:  10%|#         | 1.27G/12.7G [00:00<?, ?B/s]

In [2]:

# Evaluation function
def evaluate(test_cases, schema, db_id):
    conn = create_sample_db()
    cursor = conn.cursor()

    exact_match_count = 0
    execution_match_count = 0

    for i, case in enumerate(test_cases, 1):
        nl = case["nl"]
        expected_sql = case["expected_sql"].strip().lower()

        # Model prediction
        raw_predicted_sql = nl_to_sql(nl, db_id, schema)
        predicted_sql = clean_predicted_sql(raw_predicted_sql).strip().lower()

        # Exact match check
        is_exact_match = predicted_sql == expected_sql
        if is_exact_match:
            exact_match_count += 1

        # Execution accuracy check
        try:
            cursor.execute(expected_sql)
            expected_result = cursor.fetchall()
            cursor.execute(predicted_sql)
            predicted_result = cursor.fetchall()
            is_execution_match = expected_result == predicted_result
        except Exception as e:
            is_execution_match = False
            print(f" Execution Error in Test {i}: {e}")

        if is_execution_match:
            execution_match_count += 1

        # Show results
        print(f"\nTest Case {i}")
        print(f"NL: {nl}")
        print(f"Expected: {expected_sql}")
        print(f"Predicted: {predicted_sql}")
        print(f"✅ Exact Match: {is_exact_match}")
        print(f"✅ Execution Match: {is_execution_match}")

    total = len(test_cases)
    print("\n--- Evaluation Summary ---")
    print(f"✅ Exact Match Accuracy: {exact_match_count}/{total} = {exact_match_count / total:.2%}")
    print(f"✅ Execution Accuracy: {execution_match_count}/{total} = {execution_match_count / total:.2%}")


**Evaluation Criteria**

In [3]:

# Define test cases
test_cases = [
    {
        "nl": "What is the total number of customers from New York?",
        "expected_sql": "SELECT COUNT(*) FROM customers WHERE city = 'new york';"
    },
    {
        "nl": "List the names of customers older than 30.",
        "expected_sql": "SELECT name FROM customers WHERE age > 30;"
    },
    {
        "nl": "How many customers live in Mumbai?",
        "expected_sql": "SELECT COUNT(*) FROM customers WHERE city = 'mumbai';"
    }
]

# Schema and DB ID
schema = "customers : id (int), name (text), city (text), age (int)"
db_id = "customers_db"

# Run evaluation
evaluate(test_cases, schema, db_id)



Test Case 1
NL: What is the total number of customers from New York?
Expected: select count(*) from customers where city = 'new york';
Predicted: select count(*) from customers where city = 'new york city'
✅ Exact Match: False
✅ Execution Match: True

Test Case 2
NL: List the names of customers older than 30.
Expected: select name from customers where age > 30;
Predicted: select name from customers where age > 30
✅ Exact Match: False
✅ Execution Match: True

Test Case 3
NL: How many customers live in Mumbai?
Expected: select count(*) from customers where city = 'mumbai';
Predicted: select count(*) from customers where city = 'mumbai'
✅ Exact Match: False
✅ Execution Match: True

--- Evaluation Summary ---
✅ Exact Match Accuracy: 0/3 = 0.00%
✅ Execution Accuracy: 3/3 = 100.00%


# New Section